In [1]:
from pathlib import Path
from tqdm import tqdm
import xarray as xr
import yaml
from PIL import Image

from convml_tt.data.dataset import TRIPLET_TILE_IDENTIFIER_FORMAT

### Make set of tile images from previously sampled locations

In [2]:
# Save directory on Sense group workspace
save_dir = "/gws/nopw/j04/sensecdt/users/flojo/data/"

# Working folder
folder = "2020_test"
data_filepath = Path(save_dir+folder)

In [3]:
tiles_dir = "/home/users/flojo/Data/Tiles"
tiles_folder = '2020_test_2400tr_128px'
tiles_filepath = Path(tiles_dir) / tiles_folder
tiles_meta_filepath = tiles_filepath / "meta"

tiles_images_filepath = tiles_filepath / "images"
tiles_images_filepath.mkdir(exist_ok=True)

In [4]:
tile_size = 128

In [5]:
with open(tiles_meta_filepath / 'tile_locations.yaml', 'r') as f:
    tile_locations = yaml.safe_load(f)

In [6]:
for month in tqdm(range(1,13)):
    for satellite in ['aqua', 'terra']:
        tiles = tile_locations[month][satellite]
        granule_folder = data_filepath / str(month) / satellite / "granule_data"
        for scene_id in tiles.keys():
            granule_data = xr.open_dataset(granule_folder / f"{scene_id}.nc")
            for tile in tiles[scene_id]:
                x_c, y_c = tile['loc']['x_c'], tile['loc']['y_c']
                triplet_id = tile['triplet_id']
                tile_type = tile['tile_type']

                tile_rgb = granule_data["true_color"].isel(x=slice(x_c-tile_size//2, x_c+tile_size//2), 
                                                            y=slice(y_c-tile_size//2, y_c+tile_size//2))
                img = Image.fromarray(tile_rgb.values)
                resized_img = img.resize((256,256), Image.Resampling.NEAREST)
                tile_identifier = TRIPLET_TILE_IDENTIFIER_FORMAT.format(triplet_id=triplet_id, 
                                                                                tile_type=tile_type)
                fn_image = f"{tile_identifier}.png"
                resized_img.save(tiles_images_filepath / fn_image)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [22:26<00:00, 112.22s/it]
